# Create Mapping between Betting Odds and Fight Outcomes

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from unidecode import unidecode
from fuzzywuzzy import fuzz
pd.set_option('display.max_columns', 500)
from zipfile import ZipFile


In [12]:
# load all datasets

# mapping of wiki fight event urls to bestfightodds urls. unique key is the wikipedia fight event url
wiki_bestfightodds_url_id_mapping= pd.read_csv("../../data/bestfightodds_data/bestfightodds_urls.csv")
wiki_bestfightodds_url_id_mapping.rename(columns = {'url':'wiki_url'}, inplace = True)
wiki_bestfightodds_url_id_mapping = \
wiki_bestfightodds_url_id_mapping.loc[(wiki_bestfightodds_url_id_mapping['fight_odds_url'] != '') & \
                                      (wiki_bestfightodds_url_id_mapping['fight_odds_url'].notna()), \
                                      ['wiki_url','fight_odds_url', '#', 'Event', 'Date']]

# load each fight outcome for each ufc event. Each row is a fight outcome (from wikipedia)
wiki_fight_outcomes = pd.read_csv("../../data/wikipedia_data/wikipedia_all_ufc_fight_outcomes.csv")


# load all odds. Since I'm just creating a mapping I only need one row per fight
zip_file = ZipFile('../../data/bestfightodds_data/straight_bets.zip')
dfs = [pd.read_csv(zip_file.open(text_file.filename)) for text_file in zip_file.infolist() 
       if text_file.filename.endswith('.csv')]
moneyline_data = pd.concat(dfs)
moneyline_data = moneyline_data[['fighter1', 'fighter2', 'url']].drop_duplicates(['fighter1', 'fighter2', 'url'])
del dfs

In [13]:
# filter to all cards since 2008
wiki_bestfightodds_url_id_mapping['Date']= pd.to_datetime(wiki_bestfightodds_url_id_mapping['Date'])
wiki_bestfightodds_url_id_mapping= wiki_bestfightodds_url_id_mapping.loc[:, ["fight_odds_url", "wiki_url", "Date"]]
wiki_fight_outcomes = pd.merge(wiki_fight_outcomes, wiki_bestfightodds_url_id_mapping, how = 'inner', 
                                left_on = ['wiki_url'], right_on = ["wiki_url"] )
wiki_fight_outcomes = wiki_fight_outcomes.loc[wiki_fight_outcomes['Date'].dt.year >= 2008, :]

wiki_bestfightodds_url_id_mapping.rename(columns = {'Date':'Card_Date'}, inplace = True)
moneyline_data = pd.merge(wiki_bestfightodds_url_id_mapping[['Card_Date', 'fight_odds_url', 'wiki_url']],
                          moneyline_data, left_on = 'fight_odds_url', right_on = 'url', validate = "1:m")
moneyline_data['Card_Date'] = pd.to_datetime(moneyline_data['Card_Date'] )
moneyline_data = moneyline_data.loc[moneyline_data.Card_Date.dt.year >= 2008, :]
del moneyline_data['url']

In [14]:
# do some name cleaning to merge fight outcomes and odds data by fighter names
wiki_fight_outcomes['winner_match'] = wiki_fight_outcomes['Winner_Cleaned'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
wiki_fight_outcomes['loser_match'] = wiki_fight_outcomes['Loser_Cleaned'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
moneyline_data['fighter1_match'] = moneyline_data['fighter1'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
moneyline_data['fighter2_match'] = moneyline_data['fighter2'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))

In [15]:
# deal with duplicates before merging by just 1 fighter name
display(moneyline_data.loc[
    moneyline_data.duplicated(subset = ['fighter1_match', 'wiki_url'], keep = False), :])
moneyline_data = \
moneyline_data.loc[~((moneyline_data.fighter2 =='Paul Felder') & (moneyline_data.wiki_url == 'https://en.wikipedia.org/wiki/UFC_223')), :]
display(moneyline_data.loc[
    moneyline_data.duplicated(subset = ['fighter2_match', 'wiki_url'], keep = False), :])

,Card_Date,fight_odds_url,wiki_url,fighter1,fighter2,fighter1_match,fighter2_match
986,2018-04-07,https://www.bestfightodds.com/events/ufc-223-n...,https://en.wikipedia.org/wiki/UFC_223,Al Iaquinta,Khabib Nurmagomedov,al iaquinta,khabib nurmagomedov
990,2018-04-07,https://www.bestfightodds.com/events/ufc-223-n...,https://en.wikipedia.org/wiki/UFC_223,Al Iaquinta,Paul Felder,al iaquinta,paul felder


,Card_Date,fight_odds_url,wiki_url,fighter1,fighter2,fighter1_match,fighter2_match


In [19]:
# merge based on both fighters' name

dfa = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['wiki_url', 'winner_match', 'loser_match'],
              right_on = ['wiki_url', 'fighter1_match', 'fighter2_match'], how = 'inner', validate = '1:1')
dfb = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['wiki_url', 'winner_match', 'loser_match'],
              right_on = ['wiki_url', 'fighter2_match', 'fighter1_match'], how = 'inner', validate = '1:1')
allmatch1 = pd.concat([dfa, dfb], sort = False)

mismatches =  pd.merge(wiki_fight_outcomes[['wiki_url', 'winner_match', 'loser_match']],
                       allmatch1[['wiki_url', 'winner_match', 'loser_match', 'fighter1_match']], 
                       how = 'left', 
                       on = ['wiki_url', 'winner_match', 'loser_match']) 
mismatches = mismatches[mismatches.fighter1_match.isna()]
del mismatches['fighter1_match']
moneyline_data1 = pd.merge(moneyline_data, allmatch1[['fighter1_match', 'fighter2_match', 'winner_match', 'wiki_url']], 
                          how = 'left', on = ['fighter1_match', 'fighter2_match', 'wiki_url'])
moneyline_data1 = moneyline_data1[moneyline_data1.winner_match.isna()]
del moneyline_data1['winner_match'], dfa, dfb

# now try merging on just one name
df1 = pd.merge(mismatches, moneyline_data1, left_on = ['wiki_url', 'winner_match'],
              right_on = ['wiki_url', 'fighter1_match'], how = 'inner', validate = '1:1')
df2 = pd.merge(mismatches, moneyline_data1, left_on = ['wiki_url', 'winner_match'],
              right_on = ['wiki_url', 'fighter2_match'], how = 'inner', validate = '1:1')
df3 = pd.merge(mismatches, moneyline_data1, left_on = ['wiki_url', 'loser_match'],
              right_on = ['wiki_url', 'fighter1_match'], how = 'inner', validate = '1:1')
df4 = pd.merge(mismatches, moneyline_data1, left_on = ['wiki_url', 'loser_match'],
              right_on = ['wiki_url', 'fighter2_match'], how = 'inner', validate = '1:1')

allmatch2 = pd.concat([df1, df2, df3, df4], sort = False)
allmatch2.drop_duplicates(['wiki_url', 'fighter1_match', 'fighter2_match', 'winner_match', 'loser_match'], inplace = True)
allmatch2 = allmatch2[~(allmatch2.duplicated(['wiki_url', 'winner_match', 'loser_match'], keep = False))]
allmatch = pd.concat([allmatch1, allmatch2], sort = False)
del df1, df2, df3, df4, allmatch1, allmatch2

mismatches =  pd.merge(wiki_fight_outcomes[['wiki_url', 'winner_match', 'loser_match']],
                       allmatch[['wiki_url', 'winner_match', 'loser_match', 'fighter1_match']], 
                       how = 'left', 
                       on = ['wiki_url', 'winner_match', 'loser_match']) 
mismatches = mismatches[mismatches.fighter1_match.isna()]
del mismatches['fighter1_match']
moneyline_data1 = pd.merge(moneyline_data1, allmatch[['fighter1_match', 'fighter2_match', 'winner_match', 'wiki_url']], 
                          how = 'left', on = ['fighter1_match', 'fighter2_match', 'wiki_url'])
moneyline_data1 = moneyline_data1[moneyline_data1.winner_match.isna()]
del moneyline_data1['winner_match']

print(allmatch.shape[0])
print(mismatches.shape[0])

4653
50


In [21]:
allmatch.head()

,WeightClass,Winner,Outcome,Loser,Method,Round,Time,Notes,Card,Winner_url,Loser_url,wiki_url,event_order,Method_Cleaned,Card_Cleaned,Winner_Cleaned,Loser_Cleaned,Champion,Interim_Champion,fight_odds_url_x,Date,winner_match,loser_match,Card_Date,fight_odds_url_y,fighter1,fighter2,fighter1_match,fighter2_match,fight_odds_url
0,Catchweight (158.5 lbs),Charles Oliveira,def,Kevin Lee,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),https://en.wikipedia.org/wiki/Charles_Oliveira,https://en.wikipedia.org/wiki/Kevin_Lee_(fighter),https://en.wikipedia.org/wiki/UFC_Fight_Night:...,1.0,Sub,Main Event,Charles Oliveira,Kevin Lee,NaN,NaN,https://www.bestfightodds.com/events/ufc-on-es...,2020-03-14,charles oliveira,kevin lee,2020-03-14,https://www.bestfightodds.com/events/ufc-on-es...,Charles Oliveira,Kevin Lee,charles oliveira,kevin lee,NaN
1,Lightweight,Francisco Trinaldo,def,John Makdessi,"Decision (unanimous) (30–27, 30–27, 29–28)",3.0,5:00,NaN,Main card (ESPN+/ESPN),https://en.wikipedia.org/wiki/Francisco_Trinaldo,https://en.wikipedia.org/wiki/John_Makdessi,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,5.0,Unanimous Decision,Main Event,Francisco Trinaldo,John Makdessi,NaN,NaN,https://www.bestfightodds.com/events/ufc-on-es...,2020-03-14,francisco trinaldo,john makdessi,2020-03-14,https://www.bestfightodds.com/events/ufc-on-es...,Francisco Trinaldo,John Makdessi,francisco trinaldo,john makdessi,NaN
2,Flyweight,Brandon Moreno,def,Jussier Formiga,"Decision (unanimous) (30–27, 29–28, 29–28)",3.0,5:00,NaN,Preliminary card (ESPN+/ESPN),https://en.wikipedia.org/wiki/Brandon_Moreno,https://en.wikipedia.org/wiki/Jussier_Formiga,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,6.0,Unanimous Decision,Prelim,Brandon Moreno,Jussier Formiga,NaN,NaN,https://www.bestfightodds.com/events/ufc-on-es...,2020-03-14,brandon moreno,jussier formiga,2020-03-14,https://www.bestfightodds.com/events/ufc-on-es...,Brandon Moreno,Jussier Formiga,brandon moreno,jussier formiga,NaN
3,Women's Strawweight,Amanda Ribas,def,Randa Markos,"Decision (unanimous) (30–26, 30–25, 30–25)",3.0,5:00,NaN,Preliminary card (ESPN+/ESPN),https://en.wikipedia.org/wiki/Amanda_Ribas,https://en.wikipedia.org/wiki/Randa_Markos,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,7.0,Unanimous Decision,Prelim,Amanda Ribas,Randa Markos,NaN,NaN,https://www.bestfightodds.com/events/ufc-on-es...,2020-03-14,amanda ribas,randa markos,2020-03-14,https://www.bestfightodds.com/events/ufc-on-es...,Amanda Ribas,Randa Markos,amanda ribas,randa markos,NaN
4,Bantamweight,Enrique Barzola,vs,Rani Yahya,"Draw (majority) (29–28, 28–28, 28–28)",3.0,5:00,NaN,Preliminary card (ESPN+/ESPN),https://en.wikipedia.org/wiki/Enrique_Barzola,https://en.wikipedia.org/wiki/Rani_Yahya,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,9.0,NaN,Prelim,Enrique Barzola,Rani Yahya,NaN,NaN,https://www.bestfightodds.com/events/ufc-on-es...,2020-03-14,enrique barzola,rani yahya,2020-03-14,https://www.bestfightodds.com/events/ufc-on-es...,Enrique Barzola,Rani Yahya,enrique barzola,rani yahya,NaN


In [114]:




display(allmatch[allmatch.duplicated(subset = ['winner_match', 'loser_match', 'wiki_url'], keep = False)])
# find mismatches
mismatches =  pd.merge(wiki_fight_outcomes[['wiki_url', 'winner_match', 'loser_match']], allmatch[['wiki_url', 'winner_match', 'loser_match', 'fighter1_match']], 
                       how = 'left', 
                       left_on = ['wiki_url', 'winner_match', 'loser_match'],
                       right_on = ['wiki_url', 'winner_match', 'loser_match'])
mismatches = mismatches[mismatches.fighter1_match.isna()]
del mismatches['fighter1_match']
print(mismatches.shape[0])

# get all possibile candidates for mismatches

possibilities = pd.merge(moneyline_data, 
                         allmatch[['wiki_url', 'fighter1_match', 'fighter2_match', 'winner_match']], 
                       how = 'left', 
                       left_on = ['wiki_url', 'fighter1_match'],
                       right_on = ['wiki_url', 'fighter1_match'])
possibilities = possibilities[possibilities.winner_match.isna()]
possibilities.rename(columns = {'fighter2_match_x':'fighter2_match'}, inplace = True)
diagnose = pd.merge(mismatches, possibilities[['fighter1_match', 'fighter2_match', 'wiki_url']], left_on = 'wiki_url', right_on = 'wiki_url')
print(diagnose.shape[0])
diagnose = diagnose.sort_values(by = ['wiki_url', 'winner_match', 'loser_match'])

In [108]:
def match_score(x, y):
    score = 0
    list1 = x.split(" ")
    list2 = y.split(" ")
    if len(list1) < len(list2):
        minlist = list1
        other = list2
    else:
        minlist = list2
        other = list1
    for x in minlist:
        score += x in other
    return score
diagnose['match_score1'] = diagnose.apply(lambda x: match_score(x['fighter1_match'], x['winner_match']), axis = 1)
diagnose['match_score2'] = diagnose.apply(lambda x: match_score(x['fighter2_match'], x['winner_match']), axis = 1)
diagnose['match_score3'] = diagnose.apply(lambda x: match_score(x['fighter1_match'], x['loser_match']), axis = 1)
diagnose['match_score4'] = diagnose.apply(lambda x: match_score(x['fighter2_match'], x['loser_match']), axis = 1)
diagnose['best_match'] = diagnose[['match_score'+str(i) for i in np.arange(1,5)]].apply("max", axis = 1)
diagnose['max_match'] = diagnose.groupby(['winner_match','loser_match', 'wiki_url'])['best_match'].transform('max')
diagnose_correct = diagnose.loc[(diagnose.best_match == diagnose.max_match) & diagnose.max_match > 0, :]

In [109]:
display(diagnose_correct)

,wiki_url,winner_match,loser_match,fighter1_match,fighter2_match,match_score1,match_score2,match_score3,match_score4,best_match,max_match
1,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,saparbek safarov,nicolae negumereanu,nick negumereanu,saparbeg safarov,0,1,1,0,1,1
31,https://en.wikipedia.org/wiki/2012_in_UFC#UFC_...,chris weidman,demian maia,chris clements,matt riddle,1,0,0,0,1,1
108,https://en.wikipedia.org/wiki/2012_in_UFC#UFC_...,chris camozzi,dustin jacoby,chris clements,matt riddle,1,0,0,0,1,1
116,https://en.wikipedia.org/wiki/UFC_147,rony jason,godofredo pepey,godofredo castro,rony mariano bezerra,0,1,1,0,1,1
169,https://en.wikipedia.org/wiki/UFC_on_FX:_Johns...,mike russow,john-olav einemo,josh neer,mike pyle,0,1,0,0,1,1
176,https://en.wikipedia.org/wiki/UFC_on_FX:_Johns...,mike russow,john-olav einemo,carlos eduardo rocha,mike pierce,0,1,0,0,1,1
204,https://en.wikipedia.org/wiki/UFC_on_FX:_Johns...,michael johnson,shane roller,demetrious johnson,ian mccall,1,0,0,0,1,1
216,https://en.wikipedia.org/wiki/UFC_on_FX:_Johns...,lavar johnson,joey beltran,demetrious johnson,ian mccall,1,0,0,0,1,1
239,https://en.wikipedia.org/wiki/UFC_on_FX:_Johns...,chris camozzi,dustin jacoby,dustin pague,jared papazian,0,0,1,0,1,1
348,https://en.wikipedia.org/wiki/2012_in_UFC#UFC_...,chris camozzi,dustin jacoby,chan sung jung,dustin poirier,0,0,0,1,1,1


In [18]:



# do some name cleaning to merge fight outcomes and odds data by fighter names
# then merge using first/last name

wiki_fight_outcomes['winner_match'] = wiki_fight_outcomes['Winner_Cleaned'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
wiki_fight_outcomes['loser_match'] = wiki_fight_outcomes['Loser_Cleaned'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
wiki_fight_outcomes['winner_match_first'] = wiki_fight_outcomes['winner_match'].apply(lambda x: x.split(" ")[0])
wiki_fight_outcomes['loser_match_first'] = wiki_fight_outcomes['loser_match'].apply(lambda x: x.split(" ")[0])
wiki_fight_outcomes['winner_match_last'] = wiki_fight_outcomes['winner_match'].apply(lambda x: x.split(" ")[-1])
wiki_fight_outcomes['loser_match_last'] = wiki_fight_outcomes['loser_match'].apply(lambda x: x.split(" ")[0-1])


moneyline_data['fighter1_match'] = moneyline_data['fighter1'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
moneyline_data['fighter2_match'] = moneyline_data['fighter2'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
moneyline_data['fighter1_match_first'] = moneyline_data['fighter1_match'].apply(lambda x: x.split(" ")[0])
moneyline_data['fighter2_match_first'] = moneyline_data['fighter2_match'].apply(lambda x: x.split(" ")[0])
moneyline_data['fighter1_match_last'] = moneyline_data['fighter1_match'].apply(lambda x: x.split(" ")[-1])
moneyline_data['fighter2_match_last'] = moneyline_data['fighter2_match'].apply(lambda x: x.split(" ")[-1])

False
False
True


In [15]:
# first first
df1 = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['fight_odds_url', 'winner_match_first', 'loser_match_first'],
              right_on = ['url', 'fighter1_match_first', 'fighter2_match_first'], how = 'inner', validate = '1:1')
df2 = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['fight_odds_url', 'winner_match_first', 'loser_match_first'],
              right_on = ['url', 'fighter2_match_first', 'fighter1_match_first'], how = 'inner', validate = '1:1')

# last last 
df3 = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['fight_odds_url', 'winner_match_last', 'loser_match_last'],
              right_on = ['url', 'fighter1_match_last', 'fighter2_match_last'], how = 'inner', validate = '1:1')
df4 = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['fight_odds_url', 'winner_match_last', 'loser_match_last'],
              right_on = ['url', 'fighter2_match_last', 'fighter1_match_last'], how = 'inner', validate = '1:1')

# first last 
df5 = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['fight_odds_url', 'winner_match_first', 'loser_match_last'],
              right_on = ['url', 'fighter1_match_first', 'fighter2_match_last'], how = 'inner', validate = '1:1')
df6 = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['fight_odds_url', 'winner_match_first', 'loser_match_last'],
              right_on = ['url', 'fighter2_match_first', 'fighter1_match_last'], how = 'inner', validate = '1:1')

# last first 
df7 = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['fight_odds_url', 'winner_match_last', 'loser_match_first'],
              right_on = ['url', 'fighter1_match_last', 'fighter2_match_first'], how = 'inner', validate = '1:1')
df8 = pd.merge(wiki_fight_outcomes, moneyline_data, left_on = ['fight_odds_url', 'winner_match_last', 'loser_match_first'],
              right_on = ['url', 'fighter2_match_last', 'fighter1_match_first'], how = 'inner', validate = '1:1')

# in some cases first and last names are switched 





In [110]:
# do some name cleaning to merge fight outcomes and odds data by fighter names

wiki_fight_outcomes['winner_match'] = wiki_fight_outcomes['Winner_Cleaned'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
wiki_fight_outcomes['loser_match'] = wiki_fight_outcomes['Loser_Cleaned'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
wiki_fight_outcomes['winner_match_parts'] = wiki_fight_outcomes['winner_match'].apply(lambda x: x.split(" "))
wiki_fight_outcomes['loser_match_parts'] = wiki_fight_outcomes['loser_match'].apply(lambda x: x.split(" "))

moneyline_data['fighter1_match'] = moneyline_data['fighter1'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
moneyline_data['fighter2_match'] = moneyline_data['fighter2'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
moneyline_data['fighter1_match_parts'] = moneyline_data['fighter1_match'].apply(lambda x: x.split(" "))
moneyline_data['fighter2_match_parts'] = moneyline_data['fighter2_match'].apply(lambda x: x.split(" "))

cols1 = ['wiki_url', 'winner_match', 'loser_match', 'winner_match_parts', 'loser_match_parts']
wiki_fight_outcome_names = wiki_fight_outcomes[cols1].drop_duplicates(cols1[0:3])
cols2 = ['wiki_url', 'fighter1_match', 'fighter2_match', 'fighter1_match_parts', 'fighter2_match_parts']
moneyline_data_names = moneyline_data[cols2].drop_duplicates(cols2[0:3])

df1 = pd.merge(wiki_fight_outcome_names, moneyline_data_names, left_on = ['wiki_url', 'winner_match'],
              right_on = ['wiki_url', 'fighter1_match'], how = 'inner', validate = '1:1')
df2 = pd.merge(wiki_fight_outcome_names, moneyline_data_names, left_on = ['wiki_url', 'winner_match'],
              right_on = ['wiki_url', 'fighter2_match'], how = 'inner', validate = '1:1')
df3 = pd.merge(wiki_fight_outcome_names, moneyline_data_names, left_on = ['wiki_url', 'loser_match'],
              right_on = ['wiki_url', 'fighter1_match'], how = 'inner', validate = '1:1')
df4 = pd.merge(wiki_fight_outcome_names, moneyline_data_names, left_on = ['wiki_url', 'loser_match'],
              right_on = ['wiki_url', 'fighter2_match'], how = 'inner', validate = '1:1')

allmatch = pd.concat([df1, df2, df3, df4])


MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

In [113]:
moneyline_data_names[moneyline_data_names.fighter1_match == 'khabib nurmagomedov'].groupby('wiki_url').size()

wiki_url
https://en.wikipedia.org/wiki/2012_in_UFC#UFC_149:_Faber_vs._Barão                                                                     2
https://en.wikipedia.org/wiki/2012_in_UFC#UFC_on_Fuel_TV:_The_Korean_Zombie_vs._Poirier                                                2
https://en.wikipedia.org/wiki/The_Ultimate_Fighter:_Brazil_2#The_Ultimate_Fighter:_Brazil_2_Finale                                     1
https://en.wikipedia.org/wiki/The_Ultimate_Fighter:_Latin_America_2#The_Ultimate_Fighter_Latin_America_2_Finale:_Magny_vs._Gastelum    1
https://en.wikipedia.org/wiki/The_Ultimate_Fighter:_Live#The_Ultimate_Fighter_15_Finale                                                2
                                                                                                                                      ..
https://en.wikipedia.org/wiki/UFC_on_Fuel_TV:_Mousasi_vs._Latifi                                                                       2
https://en.wikipedia.org/wiki/UF

In [ ]:
mismatches_m21 = pd.merge(mismatches, moneyline_data, how = 'left', left_on = 'wiki_url', right_on = 'wiki_url' )

mismatches_m21['fighter1_winner_score'] = mismatches_m21.apply(lambda x: match_score(x['fighter1_match_parts'], x['winner_match_parts']))
mismatches_m21['fighter1_loser_score'] = mismatches_m21.apply(lambda x: match_score(x['fighter1_match_parts'], x['loser_match_parts']))
mismatches_m21['fighter2_winner_score'] = mismatches_m21.apply(lambda x: match_score(x['fighter2_match_parts'], x['winner_match_parts']))
mismatches_m21['fighter2_loser_score'] = mismatches_m21.apply(lambda x: match_score(x['fighter2_match_parts'], x['loser_match_parts']))
mismatches_m21['score'] = mismatches[['fighter1_winner_score', 'fighter1_loser_score', 
                                           'fighter2_winner_score', 'fighter2_loser_score'].apply("max", axis = 0)
                                          
mismatches_m21['match_type'] = mismatches[['fighter1_winner_score', 'fighter1_loser_score', 
                                           'fighter2_winner_score', 'fighter2_loser_score'].apply(pd.idxmax, axis = 0)
mismatches_m21['best_score'] = mismatches_m21.groupby(['Bet', 'fight_odds_url', 'fighter1_match', 'fighter2_match'])['score'].transform('max')
mismatches_m21 = mismatches_m21.loc[mismatches_m21['best_score'] == mismatches_m21['score'], :]                                       

In [66]:
1 +('a' in ['a', 'b'])

2

In [5]:
# do some name cleaning to make this merge easier
wiki_fight_outcomes['winner_clean'] = wiki_fight_outcomes['Winner'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
wiki_fight_outcomes['loser_clean'] = wiki_fight_outcomes['Loser'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
wiki_fighters = pd.concat([wiki_fight_outcomes[['winner_clean', 'fight_odds_url']], 
                          wiki_fight_outcomes[['loser_clean', 'fight_odds_url']]], axis = 0, sort = False)
wiki_fighters['name'] = np.where(wiki_fighters['winner_clean'].isna(),
                                 wiki_fighters['loser_clean'],
                                 wiki_fighters['winner_clean'])


moneyline_data['fighter1_clean'] = moneyline_data['fighter1'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
moneyline_data['fighter2_clean'] = moneyline_data['fighter2'].apply(lambda x: unidecode(x.lower().replace('.', '').strip()))
moneyline_fighters = pd.concat([moneyline_data[['fighter1_clean', 'url']], 
                                moneyline_data[['fighter2_clean', 'url']]], axis = 0, sort = False)
moneyline_fighters['bet_name'] = np.where(moneyline_fighters['fighter1_clean'].isna(),
                                          moneyline_fighters['fighter2_clean'],
                                          moneyline_fighters['fighter1_clean'])

merge_money_fighters = moneyline_fighters[['bet_name', 'url']].drop_duplicates()
in_wiki_no_match = pd.merge(wiki_fighters,merge_money_fighters, how = 'left', left_on = ['fight_odds_url', 'name'], right_on = ['url', 'bet_name'])
in_wiki_no_match = in_wiki_no_match.loc[in_wiki_no_match['bet_name'].isna() , ['fight_odds_url', 'name']]
print("number of names that don't match:", in_wiki_no_match['name'].nunique())

# use fuzzy matching to get matches
in_wiki_no_match_fuzzy = pd.merge(in_wiki_no_match, merge_money_fighters, how = 'left', left_on = ['fight_odds_url'], right_on = ['url'])
in_wiki_no_match_fuzzy['match_score'] = in_wiki_no_match_fuzzy.apply(lambda x: fuzz.ratio(x['name'], x['bet_name']), axis = 1)
bestscore = in_wiki_no_match_fuzzy.groupby(['name', 'fight_odds_url'])['match_score'].transform('max')
in_wiki_no_match_fuzzy_best = in_wiki_no_match_fuzzy.loc[in_wiki_no_match_fuzzy.match_score == bestscore, 
                                                        ['fight_odds_url', 'name', 'bet_name', 'match_score']].sort_values('name')
in_wiki_no_match_fuzzy_best.to_csv("../../datasets/name_bestmatches.csv", index = 'False')

number of names that don't match: 130


In [6]:
# load manual mapping from best fight odds to wiki name obtained from manually going through name_bestmatches.csv
name_mapping = pd.read_csv('../../datasets/fight_odds_to_wiki_fighter_name.csv')

# fix names

wiki_resolve_multiple_names = {'yadong song':'song yadong',
'xiaonan yan':'yan xiaonan',
'markus perez':'markus perez echeimberg',
'dong hyun ma':'dong hyun kim',
'khalil rountree':'khalil rountree jr',
'luis henrique da silva':'henrique da silva',
'joseph duffy':'joe duffy',
'joseph duffey':'joe duffy', 
'joe duffey':'joe duffy',    
'yuta sasaki':'ulka sasaki',
'liu pingyuan':'pingyuan liu',                              
'marco polo reyes':'polo reyes',
'jose quinonez':'jose alberto quinonez',
'anying wang':'wang anying',
'luiz garagorri':'luiz eduardo garagorri',
'eduardo garagorri':'luiz eduardo garagorri',
'mike graves':'michael graves'}




wiki_fight_outcomes['winner_clean'] = wiki_fight_outcomes['winner_clean'].replace(wiki_resolve_multiple_names)
wiki_fight_outcomes['loser_clean'] = wiki_fight_outcomes['loser_clean'].replace(wiki_resolve_multiple_names)
moneyline_data['fighter1_clean'] = moneyline_data['fighter1_clean'].replace(dict(zip(name_mapping.best_fight_odds_name, name_mapping.wiki_name))) 
moneyline_data['fighter2_clean'] = moneyline_data['fighter2_clean'].replace(dict(zip(name_mapping.best_fight_odds_name, name_mapping.wiki_name))) 

In [7]:
# create keys to merge fight outcomes with moneyline data
def keygen(x, a, b):
    return x[a] + " : " + x[b] if x[a] < x[b] else x[b] + " : " + x[a]

wiki_fight_outcomes['outcomes_key'] = wiki_fight_outcomes.apply(lambda x: keygen(x, 'winner_clean', 'loser_clean'), axis =1) 
moneyline_data['moneyline_key'] = moneyline_data.apply(lambda x: keygen(x, 'fighter1_clean', 'fighter2_clean'), axis = 1) 

moneyline_w_outcomes = pd.merge(moneyline_data, wiki_fight_outcomes, how = 'left', 
                                left_on = ['moneyline_key', 'url'],
                                right_on = ['outcomes_key', 'fight_odds_url'],
                                validate = "m:1")
# delete unnecessary columns
delcols = ['fight_odds_url_x',
           'fight_odds_url_y', 'wiki_url_x','wiki_url_y', 'moneyline_key',
           'outcomes_key', 'Date_x', 'Date_y', 'Winner', 'Loser']
for col in delcols:
    del moneyline_w_outcomes[col]
moneyline_w_outcomes.head()

,fighter1,fighter2,bet_type,Bet,5Dimes,BetDSI,BookMaker,SportBet,Bet365,Bovada,Sportsbook,William_H,Pinnacle,SportsInt,BetOnline,Intertops,url,bet_both_sides_group,fighter1_clean,fighter2_clean,WeightClass,Outcome,Method,Round,Time,Notes,Card,Champion,Interim_Champion,wiki_url,Date,winner_clean,loser_clean
0,Charles Oliveira,Kevin Lee,Ordinary,Charles Oliveira,2.170000,1.833333,2.140000,2.200000,2.250000,2.050000,2.150000,2.100000,2.290000,2.200000,2.100000,2.150000,https://www.bestfightodds.com/events/ufc-on-es...,0,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee
1,Charles Oliveira,Kevin Lee,Ordinary,Kevin Lee,1.729927,1.833333,1.709220,1.746269,1.657895,1.740741,1.689655,1.729927,1.684932,1.645161,1.769231,1.689655,https://www.bestfightodds.com/events/ufc-on-es...,0,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee
2,Charles Oliveira,Kevin Lee,Prop,Over 2½ rounds,2.200000,2.200000,2.310000,2.200000,NaN,NaN,NaN,2.200000,NaN,2.300000,2.250000,2.150000,https://www.bestfightodds.com/events/ufc-on-es...,1,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee
3,Charles Oliveira,Kevin Lee,Prop,Under 2½ rounds,1.714286,1.709220,1.606061,1.714286,NaN,NaN,NaN,1.621118,NaN,1.588235,1.645161,1.689655,https://www.bestfightodds.com/events/ufc-on-es...,1,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee
4,Charles Oliveira,Kevin Lee,Prop,Fight goes to decision,4.100000,NaN,NaN,4.100000,3.750000,NaN,NaN,4.000000,NaN,NaN,4.600000,NaN,https://www.bestfightodds.com/events/ufc-on-es...,2,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee


In [8]:
ordinary_bets = moneyline_w_outcomes.loc[moneyline_w_outcomes['bet_type'] == 'Ordinary', :] 
bet_clean = np.where(ordinary_bets.Bet == ordinary_bets.fighter1,ordinary_bets.fighter1_clean, ordinary_bets.fighter2_clean)
bet_won = np.where((bet_clean == ordinary_bets.winner_clean) & 
                   ((ordinary_bets.Outcome.values == 'def.') | (ordinary_bets.Outcome.values == 'def')),1,0)
moneyline_w_outcomes['bet_won'] = pd.Series(bet_won, index = ordinary_bets.index)
del moneyline_w_outcomes['fighter1']
del moneyline_w_outcomes['fighter2']
moneyline_w_outcomes.loc[ordinary_bets.index, 'Bet'] = bet_clean
moneyline_w_outcomes.head()

,bet_type,Bet,5Dimes,BetDSI,BookMaker,SportBet,Bet365,Bovada,Sportsbook,William_H,Pinnacle,SportsInt,BetOnline,Intertops,url,bet_both_sides_group,fighter1_clean,fighter2_clean,WeightClass,Outcome,Method,Round,Time,Notes,Card,Champion,Interim_Champion,wiki_url,Date,winner_clean,loser_clean,bet_won
0,Ordinary,charles oliveira,2.170000,1.833333,2.140000,2.200000,2.250000,2.050000,2.150000,2.100000,2.290000,2.200000,2.100000,2.150000,https://www.bestfightodds.com/events/ufc-on-es...,0,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee,1.0
1,Ordinary,kevin lee,1.729927,1.833333,1.709220,1.746269,1.657895,1.740741,1.689655,1.729927,1.684932,1.645161,1.769231,1.689655,https://www.bestfightodds.com/events/ufc-on-es...,0,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee,0.0
2,Prop,Over 2½ rounds,2.200000,2.200000,2.310000,2.200000,NaN,NaN,NaN,2.200000,NaN,2.300000,2.250000,2.150000,https://www.bestfightodds.com/events/ufc-on-es...,1,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee,NaN
3,Prop,Under 2½ rounds,1.714286,1.709220,1.606061,1.714286,NaN,NaN,NaN,1.621118,NaN,1.588235,1.645161,1.689655,https://www.bestfightodds.com/events/ufc-on-es...,1,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee,NaN
4,Prop,Fight goes to decision,4.100000,NaN,NaN,4.100000,3.750000,NaN,NaN,4.000000,NaN,NaN,4.600000,NaN,https://www.bestfightodds.com/events/ufc-on-es...,2,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee,NaN


In [9]:
#check if any UFC fights weren't mapped to betting odds
a = moneyline_w_outcomes[(moneyline_w_outcomes.Card.notna())& (moneyline_w_outcomes.bet_type == 'Ordinary')].shape[0]/2.0
b = wiki_fight_outcomes.shape[0]
print(a)
print(b)
print(a/b)
print(b-a)
mismatches = pd.merge(moneyline_data, wiki_fight_outcomes, how = 'right', 
                                left_on = ['moneyline_key', 'url'],
                                right_on = ['outcomes_key', 'fight_odds_url'])
mismatches = mismatches[mismatches['moneyline_key'].isna()][['winner_clean','loser_clean', 'wiki_url']]
mismatches.head()


3396.0
3397
0.9997056226081837
1.0


,winner_clean,loser_clean,wiki_url
163682,kiichi kunimoto,luiz dutra jr,https://en.wikipedia.org/wiki/UFC_Fight_Night:...


In [10]:

moneyline_w_outcomes.to_csv('../../datasets/moneyline_w_outcomes.csv', index = False)